In [ ]:
TÉCNICA | VENTAJAS BRISCAS | DESVENTAJAS
Monte Carlo | Maneja incertidumbre | Requiere más recursos
Redes Neuronales | Aprende patrones complejos | Necesita grandes datasets
Minimax | Óptimo teóricamente | No escala bien
Monte Carlo | Simula múltiples escenarios | Puede ser lento
Q-learning | Aprende de la experiencia | Requiere exploración

In [1]:
import random
from collections import defaultdict

VALORES = {
    1: 11, 3: 10, 12: 4, 11: 3, 10: 2,
    7: 0, 6: 0, 5: 0, 4: 0, 2: 0
}

PALOS = ['oros', 'copas', 'espadas', 'bastos']

class Carta:
    def __init__(self, valor, palo):
        self.valor = valor
        self.palo = palo
        self.puntos = VALORES[valor]
    
    def __repr__(self):
        return f"{self.valor} de {self.palo}"

class BriscasGame:
    def __init__(self, jugadores):
        self.jugadores = jugadores
        self.mazo = self._crear_mazo()
        self.vida = None
        self.mano_actual = []
        self.puntos = defaultdict(int)
        self.turno = 0
        
    def _crear_mazo(self):
        return [Carta(v, p) for v in VALORES for p in PALOS for _ in range(2)]
    
    def repartir(self):
        random.shuffle(self.mazo)
        self.vida = self.mazo.pop()
        for _ in range(3):
            for jugador in self.jugadores:
                jugador.recibir_carta(self.mazo.pop())
    
    def jugar_turno(self):
        jugadas = []
        for _ in range(len(self.jugadores)):
            jugador = self.jugadores[self.turno % len(self.jugadores)]
            carta = jugador.seleccionar_carta(self)
            self.mano_actual.append(carta)
            self.turno += 1
        
        ganador = self.determinar_ganador_mano()
        self.puntos[ganador] += sum(c.puntos for c in self.mano_actual)
        self.mano_actual = []
        return ganador
    
    def determinar_ganador_mano(self):
        lider_palo = self.mano_actual[0].palo
        cartas_vida = [c for c in self.mano_actual if c.palo == self.vida.palo]
        
        if cartas_vida:
            return max(enumerate(cartas_vida), 
                      key=lambda x: (x[1].valor, x[0]))[0]
        else:
            return max(enumerate([c for c in self.mano_actual 
                                if c.palo == lider_palo]),
                      key=lambda x: (x[1].valor, x[0]))[0]

class AgenteMCTS:
    def __init__(self, nombre):
        self.nombre = nombre
        self.mano = []
    
    def recibir_carta(self, carta):
        self.mano.append(carta)
    
    def seleccionar_carta(self, juego):
        simulaciones = 100
        mejores_puntos = -1
        mejor_carta = None
        
        for carta in self.mano:
            puntos = 0
            for _ in range(simulaciones):
                copia_juego = self.simular_jugada(carta, juego)
                puntos += copia_juego.puntos[self]
            
            if puntos > mejores_puntos:
                mejores_puntos = puntos
                mejor_carta = carta
        
        self.mano.remove(mejor_carta)
        return mejor_carta
    
    def simular_jugada(self, carta_inicial, juego_actual):
        # Implementación de simulación Monte Carlo
        copia_juego = BriscasGame([self] + juego_actual.jugadores[1:])
        copia_juego.vida = juego_actual.vida
        copia_juego.mano_actual = [carta_inicial]
        
        # Simular jugadas aleatorias restantes
        while len(copia_juego.mano_actual) < len(copia_juego.jugadores):
            jugador_actual = copia_juego.jugadores[copia_juego.turno % len(copia_juego.jugadores)]
            if jugador_actual == self:
                continue
            carta = random.choice(jugador_actual.mano)
            jugador_actual.mano.remove(carta)
            copia_juego.mano_actual.append(carta)
        
        return copia_juego

class AgenteHumano:
    def __init__(self, nombre):
        self.nombre = nombre
        self.mano = []
    
    def recibir_carta(self, carta):
        self.mano.append(carta)
    
    def seleccionar_carta(self, juego):
        print(f"\nTu mano: {self.mano}")
        print(f"Vida actual: {juego.vida}")
        idx = int(input("Selecciona carta (0-2): "))
        return self.mano.pop(idx)

# Ejemplo de uso
if __name__ == "__main__":
    humano = AgenteHumano("Jugador 1")
    ia = AgenteMCTS("IA Brisquera")
    juego = BriscasGame([humano, ia])
    juego.repartir()
    
    for _ in range(10):
        ganador = juego.jugar_turno()
        print(f"\nGanador de la mano: {juego.jugadores[ganador].nombre}")
        print(f"Puntos acumulados: {dict(juego.puntos)}")



Tu mano: [10 de espadas, 1 de bastos, 5 de espadas]
Vida actual: 4 de copas


KeyboardInterrupt: 

In [ ]:
class AgenteMejorado(AgenteMCTS):
    def simular_jugada(self, carta_inicial, juego_actual):
        # Implementar poda alfa-beta o funciones de evaluación heurísticas
        if self._es_ultima_mano():
            return self._evaluar_mano_final()
        # Añadir lógica de aprendizaje por refuerzo


In [ ]:
import tensorflow as tf

modelo = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(estado_size,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

modelo.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
